# Python ORM

This notebook will show you how the sqlalchemy library can be used to provide an object relational mapper for python classes and sqlite sql database files.

In [19]:
pip install --upgrade Perdy

  Running setup.py install for jsonpath ... -\done
  Found existing installation: Perdy 1.3
    Uninstalling Perdy-1.3:
      Successfully uninstalled Perdy-1.3
  Running setup.py install for Perdy ... -\|/-\done
Note: you may need to restart the kernel to use updated packages.


In [2]:
#!/usr/bin/env python3

import os, re, sys, argparse, json, datetime, logging

from io import StringIO

from Perdy.pretty import prettyPrintLn

## sqlalchemy

The most excellent ORM available for python is the sqlalchemy library

In [20]:
pip install --upgrade sqlalchemy

     |████████████████████████████████| 6.1MB 5.5MB/s /s eta 0:00:01
  Found existing installation: SQLAlchemy 1.3.13
    Uninstalling SQLAlchemy-1.3.13:
      Successfully uninstalled SQLAlchemy-1.3.13
  Running setup.py install for sqlalchemy ... -\|/-\|/-\|/-\|/-\|/-\|/done
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sqlalchemy
import sqlalchemy.orm
from sqlalchemy import \
    Column, \
    Integer, \
    String, \
    ForeignKey
from sqlalchemy.engine import \
    reflection
from sqlalchemy.orm import \
    relationship, \
    backref
from sqlalchemy.orm.collections import \
    InstrumentedList, \
    InstrumentedDict, \
    InstrumentedSet, \
    MappedCollection
from sqlalchemy.ext.declarative import \
    declarative_base

## jsonweb

to serialise python classes we are using jsonweb decorator, which allows you to easily render python classes in json

In [22]:
pip install --upgrade jsonweb

Requirement already up-to-date: jsonweb in /private/var/mobile/Containers/Data/Application/4053D624-71DA-49F6-96B2-782AE93B4AC1/Library/lib/python3.7/site-packages (0.8.2)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from jsonweb.encode import \
    to_object, \
    dumper
from jsonweb.decode import \
    from_object, \
    loader

In [7]:
# the following shippet shows visible items in the dir() of the module
print(json.dumps(sorted(list(filter(lambda x: not x.startswith('_'), dir()))), indent=4))

[
    "Column",
    "ForeignKey",
    "In",
    "InstrumentedDict",
    "InstrumentedList",
    "InstrumentedSet",
    "Integer",
    "MappedCollection",
    "Out",
    "String",
    "StringIO",
    "argparse",
    "backref",
    "datetime",
    "declarative_base",
    "dumper",
    "exit",
    "from_object",
    "get_ipython",
    "json",
    "loader",
    "logging",
    "os",
    "prettyPrintLn",
    "quit",
    "re",
    "reflection",
    "relationship",
    "sqlalchemy",
    "sys",
    "to_object"
]


## quietly

to hide some of the logging noise lets set the level to warning for those noisy critters

In [8]:
def quietly():
    for logger in [
        "sqlalchemy.orm.relationships.RelationshipProperty",
        "sqlalchemy.orm.strategies.LazyLoader", "sqlalchemy.orm.path_registry",
        "sqlalchemy.orm.mapper.Mapper", "sqlalchemy.engine.base.Engine",
        "sqlalchemy.pool.NullPool", "sqlalchemy.pool.impl.NullPool",
        "sqlalchemy.pool.SingletonThreadPool",
        "requests.packages.urllib3.connectionpool"
    ]:
        print(logger)
        logging.getLogger(logger).setLevel(logging.WARNING)
        
quietly()

sqlalchemy.orm.relationships.RelationshipProperty
sqlalchemy.orm.strategies.LazyLoader
sqlalchemy.orm.path_registry
sqlalchemy.orm.mapper.Mapper
sqlalchemy.engine.base.Engine
sqlalchemy.pool.NullPool
sqlalchemy.pool.impl.NullPool
sqlalchemy.pool.SingletonThreadPool
requests.packages.urllib3.connectionpool


## Base

sqlalchemy uses a Base class to allow objects to be marshalled and unmarshalled from the database

### MyBase

to showcase how we can implement inheritance in an SQL ORM, I have chosen to create a MyBase class that stores an ID for the classes, this saves having to do it each time for each sub class. It also shows how we use an "inherited" attribute to define the "type" of the subclass

In [9]:
Base = declarative_base()

class MyBase(Base):

    __tablename__ = 'base'
    id            = Column(Integer, primary_key=True)
    inherited     = Column(String(256))

    __mapper_args__ = {
        'polymorphic_identity' : 'base',
        'polymorphic_on' : inherited
    }

    def __init__(self,
        id=None,
        inherited=None
    ):
        self.id = id
        self.inherited=inherited
        return

    def __dir__(self):
        return [
            'id'
        ]
    
print(json.dumps(sorted(list(filter(lambda x: not x.startswith('_'), dir(MyBase)))), indent=4))

[
    "id",
    "inherited",
    "metadata"
]


## Address

lets define a class called Address that extends our polymorphic base class

### base.id

because our *Address* class extgends the *MyBase* class, we have to use a *ForeignKey* to link the parent to the child

### ForeignKey

we are setting up an sql foreign key to be used to link between the Address class and the forthcomming User class.

### super

please note that we are ensuring to call the super __init__ method of the base class

In [10]:
@from_object()
@to_object()
class Address(MyBase):

    __tablename__ = 'address'
    id            = Column(Integer,    ForeignKey('base.id'), primary_key=True)
    email         = Column(String(50), nullable=False)
    user_id       = Column(Integer,    ForeignKey('user.id'))

    __mapper_args__ = {
        'polymorphic_identity':'address'
    }

    def __init__(self,
        id=None,
        inherited='address',
        email=None,
        user_id=None
    ):
        super(Address,self).__init__(
            id=id,
            inherited=inherited
        )
        self.email=email
        self.user_id=user_id
        return

    def __dir__(self):
        return [
            'id',
            'email'
        ]
    
print(json.dumps(sorted(list(filter(lambda x: not x.startswith('_'), dir(Address)))), indent=4))

[
    "email",
    "id",
    "inherited",
    "metadata",
    "user_id"
]


## User

To show a one to many relationship, we create a User class type that has a *relationship* with the *Address* class.

In [11]:
@from_object()
@to_object()
class User(MyBase):

    __tablename__ = 'user'
    id            = Column(Integer,         ForeignKey('base.id'), primary_key=True)
    name          = Column(String(50))
    fullname      = Column(String(25))
    password      = Column(String(25))
    addresses     = relationship("Address", uselist=True, foreign_keys=[Address.user_id])

    __mapper_args__ = {
        'polymorphic_identity':'user'
    }

    def __init__(self,
        id=None,
        inherited='user',
        name=None,
        fullname=None,
        password=None,
        addresses=[]
    ):
        super(User,self).__init__(
            id=id,
            inherited=inherited
        )
        self.name=name
        self.fullname=fullname
        self.password=password
        self.addresses=addresses
        return

    def __dir__(self):
        return [
            'id',
            'name',
            'fullname',
            'password',
            'addresses'
        ]
    
print(json.dumps(sorted(list(filter(lambda x: not x.startswith('_'), dir(User)))), indent=4))

[
    "addresses",
    "fullname",
    "id",
    "inherited",
    "metadata",
    "name",
    "password"
]


## deploy

The *engine* is used to setup an ORM relationship with the classes.
To create a database from scratch, we use the *Base.metadata.create_all* method to build a new sqlite database fromt he ORM model.

Note we can use any number of database technologies, please see the [sqlalchemy](https://sqlalchemy.org) site for more detail.

In [12]:
def deploy(url, database, make=False, verbose=False):
    if make:
        if os.path.isfile(database):
            os.unlink(database)

    engine = sqlalchemy.create_engine('%s/%s'%(url,database), echo=verbose)

    if make:
        Base.metadata.create_all(engine)

    return engine

## reflect

We have created a help class to instanciate a Session object that is used to transact with the database, as part of this we have thrown in some reflection allowing you to intraspect the ORM model programatically to see what is already in an existing database.

In [13]:
def reflect(engine, verbose=False):
    inspector = reflection.Inspector.from_engine(engine)
    if verbose:    
        print(inspector)
        for table in inspector.get_table_names():
            print(table)
            for column in inspector.get_columns(table):
                print('\t%s : %s'%(column['name'], column['type']))

    Session = sqlalchemy.orm.sessionmaker()
    Session.configure(bind=engine)
    return Session

## insert

To insert data we make use of a *session* which we will show later to create a python class object, fill it with values, then add it to the sqlalchemy session and commit the changes.

In [23]:
def insert(session, verbose=False):
    user = User(
        name='dedson',
        fullname='David Edson',
        password='abc123'
    )
    user.addresses = [
        Address(
            email='eddo888@tpg.com.au'
        ),
        Address(
            email='david.edson@gmail.com'
        )
    ]
    session.add(user)
    session.commit()
    if verbose:
        print('saved user.id=%s'%user.id)
    return user.id

## query

To show one of they many ways we can retrieve or query data from the ORM, we make use of the *session* to query based on a *User* class type, and then filter based an attribute name=value pattern.

In [15]:
def query(session, verbose=False):
    users = session.query(User).filter_by(name='dedson').all()
    js = json.loads(dumper(users))
    if verbose:
        print(json.dumps(js, indent=4))
    return js

## setup

Now that we have all the definitions and scaffoldoign in place, lets actually connect to the datavbase and create a Session to play with.

In [16]:
url = 'sqlite://'
database = 'mySqlAlchemy.db'
print(database)

engine = deploy(url, database, make=True)
Session = reflect(engine, verbose=False)

mySqlAlchemy.db


## Session()

Instantiate a Session object and use it to insert a record.

In [17]:
session = Session()
insert(session, verbose=True)
session.close()

saved user.id=1


## query

Create a new session object and use it to query the database.

In [18]:
session = Session()
js = query(session, verbose=False)
session.close()

prettyPrintLn(js)

[
  {
    "addresses" : [
      {
        "email" : "eddo888@tpg.com.au",
        "id" : 2,
        "__type__" : "Address"
      },
      {
        "email" : "david.edson@gmail.com",
        "id" : 3,
        "__type__" : "Address"
      }
    ],
    "fullname" : "David Edson",
    "id" : 1,
    "name" : "dedson",
    "password" : "abc123",
    "__type__" : "User"
  }
]


# Conclusion

I hope this simple example gives you a taste for using python and sqlalchemy to crate an ORM and start using a normalised class based interface to your relational databases.